In [21]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [22]:
# connection to Postgres 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-26 08:06:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2021-12-26 08:06:39 (1.31 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [23]:
from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [25]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------+-------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                               |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                                     |review_body                                                                                            |review_date|
+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+---------------

In [26]:
# Count the number of records (rows) in the dataset.
df.count()

149086

In [27]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [28]:
from pyspark.sql.functions import to_date

In [29]:
df = df.withColumn('review_date',to_date(df.review_date, 'yyyy-mm-dd'))
df = df.withColumn("customer_id",df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent",df["product_parent"].cast(IntegerType()))
df = df.withColumn("star_rating",df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes",df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes",df["total_votes"].cast(IntegerType()))

In [30]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [31]:
# **Transform** the dataset to fit the tables in the [schema file](../Resources/schema.sql). Be sure the DataFrames match in data type and in column name.

In [32]:
review_id_table = df.select("review_id", "customer_id", "product_id","product_parent","review_date")
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806| 2015-01-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287| 2015-01-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283| 2015-01-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283| 2015-01-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [33]:
products = df.select("product_id","product_title").dropDuplicates(["product_id"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0002CZPPG|Apple iTunes Prep...|
|B000LGKQHU|Apple iTunes $25 ...|
|B001GXRQW0|Amazon.com Gift C...|
|B001H324JO|Amazon.com Gift C...|
|B001H53QDK|Amazon.com Gift C...|
+----------+--------------------+
only showing top 5 rows



In [34]:
customers= df.groupBy("customer_id").count().withColumnRenamed("count","customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12819130|             2|
|   50079826|             2|
|    3399669|             1|
|   23344898|             1|
|   42139031|             1|
+-----------+--------------+
only showing top 5 rows



In [35]:
vine_table = df.select("review_id", "star_rating", "helpful_votes","total_votes","vine")
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [36]:
# **Load** the DataFrames that correspond to tables into an RDS instance. **Note:** This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.

In [37]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:<endpoint>:<port>/<db_name>"
config = {"user":"<user>", 
          "password": "<pass>", 
          "driver":"org.postgresql.Driver"}


In [38]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [39]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [40]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [41]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)